# Accessing the Omeka S API

Some examples for an initial exploration of the Omeka S API.
For reference, the Omeka S documentation can be accessed at https://omeka.org/s/docs/developer/api/rest_api/.
See [this post](https://forum.omeka.org/t/example-api-usage-using-curl/8083) on using `curl` to query the API.

While any public objects can be accessed or viewed in the API, private data and write or modify
options require an access key. In these examples, the access key is in `../collection-project/omeka-credentials.json`.

## Setup

In [ ]:
import requests
import json

In [ ]:
# establish API location/endpoints

siteUrl = 'http://jajohnst.si676.si.umich.edu/omeka-s' # if you replicate this example, provide the URI for your site
endpoint = '/api'

### Get Credentials

Not necessary for most operations, but it is useful to see one way to address this process.

In [ ]:
def get_credentials(credential_file_path):
    '''Retrieve Omeka S Api credentials from another file. 
    That file must be a JSON file.'''

    with open(credential_file_path, 'r') as credentials:
        keys = json.load(credentials)
    
    return keys['key_identity'], keys['key_credential']

In [ ]:
# retrieve API credentials

credential_file_path = '../collection-site-materials/omeka-credentials.json'

key_identity, key_credential = get_credentials(credential_file_path)

print('key_identity:',key_identity,'\nkey_credential:',key_credential)

## Get item list

This operation uses the `items` actions from the API.

First, assemble the URI:

In [ ]:
action = '/items'

r_url = siteUrl + endpoint + action

print(r_url)

Then, set up parameters:

In [ ]:
parameters = {
    'key_credential': key_credential,
    'key_identity':   key_identity,
    'pretty_print':   1,
    'format':         'jsonld'
}

In [ ]:
r = requests.get(r_url, params=parameters)

print(r.url)
print(r.status_code)

Take a look at the response (using requests built-in `json` parser):

In [ ]:
len(r.json())

In [ ]:
for element in r.json():
    print(element)

In [ ]:
for element in r.json():
    print(element['o:id'],':',element['o:title'])

### Get the information about just one item

In [ ]:
item_id = r.json()[0]['o:id']
print(item_id)

Construct a new URI:

In [ ]:
r_url = siteUrl + endpoint + action + '/' + str(item_id)

Make the request:

In [ ]:
r = requests.get(r_url, params=parameters)

print(r.url)
print(r.status_code)

In [ ]:
r.json()

Save the item information locally as a json file

In [ ]:
with open(f'item-{item_id}-metadata.json', 'w') as f:
    f.write(json.dumps(r.json(), indent=2))
    print('wrote metadata to local file')

## Retrieve multiple items, using a feeder list

Based on the full inventory from the `api/items` action, retrieve individual items:

In [ ]:
# get all items
action = '/items'

r_url = siteUrl + endpoint + action

r = requests.get(r_url, params=parameters)

print(r.url)
print(r.status_code)

In [ ]:
# record each item id in a list
item_list = list()

for item in r.json():
    item_list.append(item['o:id'])

print(item_list)

In [ ]:
# retrieve the individual item records and save to a file

count = 0

for item in item_list:
    r_url = siteUrl + endpoint + action + '/' + str(item)
    print(f'requesting {r_url}')
    r = requests.get(r_url, params=parameters)
    fname = 'item-' + str(item) + '-metadata.json'
    with open(fname, 'w') as f:
        f.write(json.dumps(r.json(), indent=2))
    print(f'saved to file {fname}')
    count += 1

print(f'\nwrote {count} new files')

## Getting the "Context"

You may have noted each of the keys in the Omeka S JSON responses have a namespace.
This is because this response is JSON-LD, or JSON linked data. The `@context` tag refers to the API endpoint where these are defined.
In this case, the endpoint is `api-context`. 

Request the context:

In [ ]:
r_url = 'http://jajohnst.si676.si.umich.edu/omeka-s/api-context'

r = requests.get(r_url, params=parameters)

with open('omeka-s-api-context.json', 'w') as f:
    f.write(json.dumps(r.json(), indent=2))

In [ ]:
r.url

In [ ]:
r.json()